In [5]:
#THIS BLOCK IS FOR IMPORTING AND READING GENERAL INFO ABOUT DATA
#Library managment is made through anaconda environments

import mne
import numpy as np
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

from scipy.signal import butter, iirnotch, sosfilt, zpk2sos
from scipy import signal

#Upload arquivo EDF
path =r'C:\Users\thaismarchetti\Documents\Doutorado\Dados\Experimentos\paired_pulse_context_tree\Preliminary_tests\TCM_05_06_2025\thais_tree.edf'
raw = mne.io.read_raw_edf(path, preload=True)

#Check if the data was loaded and give general info about it
print(raw.info)
print("Data channels available: ",raw.ch_names)

#PLOT THE RAW DATA CORRESPONDING TO THE EMG CHANNELS (Don't forget to change channel name)
#raw.plot(picks=['Input 33'], scalings='auto', title='Raw EEG Data', show=True, remove_dc=False) #EEG plot, adds a variable DC level, further investigation is needed
data, times = raw.copy().pick(['Input 33']).get_data(return_times=True)

plt.plot(times, data[0], color='black')
plt.grid(True)
plt.show()

Extracting EDF parameters from C:\Users\thaismarchetti\Documents\Doutorado\Dados\Experimentos\paired_pulse_context_tree\Preliminary_tests\TCM_05_06_2025\thais_tree.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 11049999  =      0.000 ...  2210.000 secs...
<Info | 8 non-empty values
 bads: []
 ch_names: Input 33
 chs: 1 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 2500.0 Hz
 meas_date: 2025-06-05 11:04:37 UTC
 nchan: 1
 projs: []
 sfreq: 5000.0 Hz
 subject_info: 5 items (dict)
>
Data channels available:  ['Input 33']


In [2]:
# THIS BLOCK APPLIES FILTERS TO THE DATA
# Apply a high-pass filter at 20 Hz  using an IIR Butterworth filter
filt_raw = raw.copy().filter(l_freq=20, h_freq=None, picks=['Input 33'], method="iir", n_jobs=2, iir_params=dict(order=8, ftype="butter"))

# Define the frequencies for the notch filter to remove powerline noise and harmonics
freqs = (60, 120, 180, 240, 300, 360)

# Apply the notch filter 
filt_raw.notch_filter(freqs=freqs, trans_bandwidth=2, notch_widths=2, picks=[0])
data, times =filt_raw[:,:] ##### Não entendi essa linha
# Extract data for the first channel

emg_data = filt_raw.get_data()
print(emg_data.shape)

data, times = filt_raw.copy().pick_channels(['Input 33']).get_data(return_times=True)
plt.plot(times, data[0], color='black')
plt.grid(True)
plt.show()


Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 20 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoff at 20.00 Hz: -6.02 dB



[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 1.00 Hz
- Upper transition bandwidth: 1.00 Hz
- Filter length: 16501 samples (3.300 s)

(1, 11050000)
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


In [3]:
# THIS BLOCK GETS THE ONSETS (TRIGGER INSTANTS) AND SEPARETES THEM IN A AND B

onsets = raw.annotations.onset
#print(raw.annotations.onset)

descriptions = raw.annotations.description
trigger_a_onsets = [onsets[i] for i in range(len(descriptions)) if "A" in descriptions[i]]
trigger_b_onsets = [onsets[i] for i in range(len(descriptions)) if "B" in descriptions[i]]

trigger_a_onsets = [float(onset) for onset in trigger_a_onsets]
trigger_b_onsets = [float(onset) for onset in trigger_b_onsets]

print("Onsets Trigger A:", trigger_a_onsets)
print(len(trigger_a_onsets))
print("Onsets Trigger B:", trigger_b_onsets)

len(trigger_a_onsets) 

emg_data = filt_raw.copy().pick('Input 33').get_data()
print("EMG data:", emg_data)
print(emg_data.shape)

Onsets Trigger A: [23.9636, 33.026, 40.984, 50.4044, 58.4644, 66.7012, 74.9568, 83.9212, 91.8514, 101.3584, 109.0416, 117.0322, 124.783, 133.3608, 142.572, 150.152, 158.1514, 166.9952, 175.7544, 184.7286, 192.6032, 200.1522, 208.7996, 216.9254, 224.9716, 234.4604, 243.4614, 251.1328, 259.3108, 267.9682, 276.5904, 285.5858, 293.124, 302.2174, 310.0802, 318.9462, 327.3868, 335.3394, 344.429, 352.6006, 361.8362, 370.1918, 379.195, 386.965, 395.9612, 405.426, 413.1222, 422.5018, 430.1544, 438.7808, 447.7596, 455.7098, 463.2952, 471.6712, 480.4622, 489.5, 497.651, 506.6724, 515.6294, 523.5056, 531.6924, 540.9608, 548.6206, 556.6606, 565.5734, 573.8692, 583.2538, 592.1062, 600.6944, 609.443, 616.9886, 625.0006, 633.525, 642.7124, 650.5756, 659.0588, 667.1314, 676.0436, 684.0564, 692.2782, 700.7466, 708.9932, 717.5426, 725.4382, 734.3616, 741.9624, 749.8372, 757.4582, 765.3766, 774.1692, 783.5964, 792.0464, 800.113, 808.7854, 817.0834, 826.3096, 834.6898, 843.7766, 851.9756, 861.3254, 870.497

In [4]:
#THIS BLOCK SEARCHES FOR THE MAXIMUM AND MINIMUN IN A TIME INTERVAL (GIVEN THE ONSETS)

artefact_escape_ms = 13 / 1000 # This is the first limit of the window, remember it should be after the artifact of the test stimulus
end_window_ms = 60 / 1000
fs=5000
maximo_emg = []
minimo_emg = []
filtered_signal= emg_data.T

for t in trigger_a_onsets:
    #This makes the window in terms of 
    indice_trigger = int((t + artefact_escape_ms) * fs)
    print(indice_trigger)
    indice_final_busca = int((t + end_window_ms) * fs)
    print(indice_final_busca)
    
   
    maximo_emg.append(np.max(filtered_signal[indice_trigger:indice_final_busca]))
    minimo_emg.append(np.min(filtered_signal[indice_trigger:indice_final_busca]))
    
    # Create a scatter plot for the start of the searching
    plt.scatter(indice_trigger, np.zeros_like(indice_trigger), color='k', marker='*', label='Trigger')

    # Create a scatter plot for the end of the searching interval
    #OBS: THE ASTERISK THAT DELIMITS THE SEARCH INTERVAL 
    plt.scatter(indice_final_busca, np.zeros_like(indice_final_busca), color='red', marker='*', label='Final Busca')

indices = np.array(raw.annotations.onset) * fs

plt.plot(filtered_signal)

for index in indices:
    plt.axvline(x=index, color='green', linestyle='--', label='Event')
   
plt.show()
   
# Converter para arrays numpy se necessário
maximo_emg = np.array(maximo_emg)
minimo_emg = np.array(minimo_emg)

print("Máximos EMG:", maximo_emg)
print("Mínimos EMG:", minimo_emg)

119883
120117
165195
165430
204985
205220
252087
252322
292387
292622
333571
333806
374849
375084
419671
419906
459322
459557
506857
507092
545273
545508
585226
585461
623980
624215
666869
667104
712925
713160
750825
751060
790822
791057
835041
835276
878837
879072
923708
923943
963081
963316
1000826
1001061
1044063
1044298
1084692
1084927
1124923
1125158
1172367
1172602
1217372
1217607
1255729
1255964
1296618
1296853
1339906
1340141
1383016
1383252
1427994
1428229
1465685
1465920
1511151
1511387
1550465
1550701
1594795
1595031
1636998
1637234
1676762
1676997
1722209
1722444
1763067
1763303
1809246
1809481
1851024
1851259
1896039
1896275
1934889
1935125
1979871
1980106
2027194
2027430
2065676
2065911
2112574
2112809
2150837
2151072
2193969
2194204
2238863
2239098
2278613
2278849
2316541
2316776
2358421
2358656
2402376
2402611
2447565
2447800
2488320
2488555
2533427
2533662
2578212
2578447
2617593
2617827
2658527
2658762
2704869
2705103
2743168
2743402
2783368
2783603
2827932
2828166
28

In [ ]:
# THIS BLOCK CONVERTS THE EMG VALUES TO MICRO VOLTS AND SAVES THE DATA
MEP=(maximo_emg-minimo_emg)*1000000
MEP
np.savetxt('MEP_microvoltsvalues_TM_tree.txt',MEP, fmt='%.5f') #Don't forget to change the file title

In [57]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

mep_tree = pd.read_csv(
    r'C:\Users\thaismarchetti\Documents\Doutorado\Dados\Experimentos\paired_pulse_context_tree\Preliminary_tests\TCM_05_06_2025\MEP_microvoltsvalues_TM_tree.txt',
    header=None,
    sep='\t'
)

sequence_tree = pd.read_csv(
    r'C:\Users\thaismarchetti\Documents\Doutorado\Dados\Experimentos\paired_pulse_context_tree\Preliminary_tests\TCM_05_06_2025\tree_sequence_300.txt',
    header=None,
    sep='\t'
)

data_tree = pd.DataFrame({
    "mean_mep": mep_tree.iloc[:, 0],
    "pulse_id": sequence_tree.iloc[:, 0]
})

# Dicionário para os rotulos dos pulsos
pulse_labels = {
    0: "Inibitório",
    1: "Pulso simples",
    2: "Excitatório"
}

data_tree["pulse_id"] = data_tree["pulse_id"].map(pulse_labels)
data_tree["pulse_id"] = pd.Categorical(data_tree["pulse_id"], categories=["Inibitório", "Pulso simples", "Excitatório"])

# === BOXPLOT ===
plt.figure(figsize=(8, 6))
sns.boxplot(x="pulse_id", y="mean_mep", data=data_tree)
plt.xlabel("Tipo de Pulso")
plt.ylabel("Média do MEP (μV)")
plt.title("Distribuição do MEP por Tipo de Pulso")
plt.tight_layout()
plt.show()


In [34]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


data_tree = pd.DataFrame({
    "mean_mep": mep_tree.iloc[:, 0],
    "pulse_id": sequence_tree.iloc[:, 0]
})


data_tree["prev_pulse_id"] = data_tree["pulse_id"].shift(1)
filtered = data_tree[data_tree["pulse_id"] == 1].copy()
print(data_tree)
print(filtered)

       mean_mep  pulse_id  prev_pulse_id
0     431.86569         1            NaN
1     830.70035         1            1.0
2      88.12154         0            1.0
3    2907.05163         2            0.0
4    2310.87213         1            2.0
..          ...       ...            ...
295         NaN         1            1.0
296         NaN         0            1.0
297         NaN         2            0.0
298         NaN         1            2.0
299         NaN         0            1.0

[300 rows x 3 columns]
       mean_mep  pulse_id  prev_pulse_id
0     431.86569         1            NaN
1     830.70035         1            1.0
4    2310.87213         1            2.0
7     501.39405         1            2.0
9     893.87070         1            0.0
..          ...       ...            ...
289         NaN         1            2.0
292         NaN         1            2.0
294         NaN         1            0.0
295         NaN         1            1.0
298         NaN         1        

In [35]:
# Mapear os rótulos dos pulsos anteriores
context_labels = {
    0: "Precedido por Inibitório",
    1: "Precedido por Pulso simples",
    2: "Precedido por Excitatório"
}

filtered["context"] = filtered["prev_pulse_id"].map(context_labels)
filtered["context"] = pd.Categorical(filtered["context"],
                                     categories=["Precedido por Inibitório", "Precedido por Pulso simples", "Precedido por Excitatório"])
print(filtered)

plt.figure(figsize=(8, 6))
sns.boxplot(x="context", y="mean_mep", data=filtered)
plt.xlabel("Contexto do Pulso Simples")
plt.ylabel("Média do MEP (μV)")
plt.title("MEP de Pulsos Simples por Tipo de Pulso Precedente")
plt.tight_layout()
plt.show()

       mean_mep  pulse_id  prev_pulse_id                      context
0     431.86569         1            NaN                          NaN
1     830.70035         1            1.0  Precedido por Pulso simples
4    2310.87213         1            2.0    Precedido por Excitatório
7     501.39405         1            2.0    Precedido por Excitatório
9     893.87070         1            0.0     Precedido por Inibitório
..          ...       ...            ...                          ...
289         NaN         1            2.0    Precedido por Excitatório
292         NaN         1            2.0    Precedido por Excitatório
294         NaN         1            0.0     Precedido por Inibitório
295         NaN         1            1.0  Precedido por Pulso simples
298         NaN         1            2.0    Precedido por Excitatório

[127 rows x 4 columns]


In [36]:
print(data_tree)

data_tree["prev_prev_pulse_id"] = data_tree["prev_pulse_id"].shift(1)
condicao1 = (data_tree["prev_pulse_id"] == 1) & (data_tree["prev_prev_pulse_id"] == 2) #Vai resultar nos 0 precedidos por (2,1)
condicao2 = (data_tree["prev_pulse_id"] == 1) & (data_tree["prev_prev_pulse_id"] == 1) #Vai resultar nos 0 precedidos por (1,1)

subset1 = data_tree[condicao1]
subset2 = data_tree[condicao2]


print(subset1.shape)
print(subset2.shape)

       mean_mep  pulse_id  prev_pulse_id
0     431.86569         1            NaN
1     830.70035         1            1.0
2      88.12154         0            1.0
3    2907.05163         2            0.0
4    2310.87213         1            2.0
..          ...       ...            ...
295         NaN         1            1.0
296         NaN         0            1.0
297         NaN         2            0.0
298         NaN         1            2.0
299         NaN         0            1.0

[300 rows x 3 columns]
(73, 4)
(27, 4)


In [ ]:
from scipy.stats import ks_2samp
data = [subset1["mean_mep"], subset2["mean_mep"]]
labels = ["Precedido por (2,1)", "Precedido por (1,1)"]

#boxplot
plt.figure(figsize=(8, 5))
plt.boxplot(data, labels=labels, patch_artist=True)

plt.ylabel("mean_mep")
plt.title("Comparação de mean_mep por sequência anterior de pulsos")
plt.grid(True, axis='y')

plt.tight_layout()
plt.show()

ks_stat, p_value = ks_2samp(subset1["mean_mep"], subset2["mean_mep"])

print(f"Estatística KS: {ks_stat}") #Não sei o q é isso
print(f"Valor p: {p_value}")

C:\Users\thaismarchetti\AppData\Local\Temp\ipykernel_28332\1382555594.py:7: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data, labels=labels, patch_artist=True)


Estatística KS: 0.2237442922374429
Valor p: 0.233187585047744


In [25]:
from scipy.stats import ks_2samp

# Filtrando os dois grupos de interesse
grupo_exitatorio = filtered[filtered["context"] == "Precedido por Excitatório"]["mean_mep"]
grupo_simples = filtered[filtered["context"] == "Precedido por Pulso simples"]["mean_mep"]

# Teste de Kolmogorov-Smirnov
ks_stat, p_value = ks_2samp(grupo_exitatorio, grupo_simples)

print(f"Estatística KS: {ks_stat}")
print(f"Valor p: {p_value}")


Estatística KS: 0.17858954845256214
Valor p: 0.4900517851602631


In [30]:
#THIS BLOCK PLOTS THE HISTOGRAM OF THE MEP AMPLITUDE
plt.figure(figsize=(10, 6))
plt.hist(MEP, bins=50, color='blue', alpha=0.7)
plt.title('Histograma do Canal EEG 001')
plt.xlabel('Amplitude')
plt.ylabel('Frequência')
plt.show()

In [92]:
import numpy as np

pre = 13/1000  
post = 60/1000

trigger_a_onsets = np.array(trigger_a_onsets) 
trigger_a_onsets = trigger_a_onsets.reshape(-1, 1)
sequence_tree = np.array(sequence_tree)

trig_seq = np.column_stack((sequence_tree[1:240], trigger_a_onsets[1:240]))
print(trig_seq)

n = trig_seq.shape[0]  

segmentos = [] 

for i in range(n):
    indice_trigger = int((trig_seq[i, 1] + pre) * fs)
    indice_final_busca = int((trig_seq[i, 1] + post) * fs)

    segmento_atual = filtered_signal[indice_trigger:indice_final_busca]*1000000
    segmentos.append(segmento_atual)

segmentos = np.array(segmentos, dtype=object)

indices_0 = np.where(trig_seq[:, 0] == 0)[0]
indices_1 = np.where(trig_seq[:, 0] == 1)[0]
indices_2 = np.where(trig_seq[:, 0] == 2)[0]

plt.figure(figsize=(10, 6))

for idx in indices_2:
    tempo = np.arange(0, len(segmentos[idx])) / fs
    plt.plot(tempo, segmentos[idx], alpha=0.5)

plt.title("Traçados dos MEP para pulsos excitatório")
plt.xlabel("Tempo (s)")
plt.ylabel("Amplitude (μV)")
plt.grid(True)
plt.show()



[[1.0000000e+00 3.3026000e+01]
 [0.0000000e+00 4.0984000e+01]
 [2.0000000e+00 5.0404400e+01]
 [1.0000000e+00 5.8464400e+01]
 [0.0000000e+00 6.6701200e+01]
 [2.0000000e+00 7.4956800e+01]
 [1.0000000e+00 8.3921200e+01]
 [0.0000000e+00 9.1851400e+01]
 [1.0000000e+00 1.0135840e+02]
 [1.0000000e+00 1.0904160e+02]
 [0.0000000e+00 1.1703220e+02]
 [2.0000000e+00 1.2478300e+02]
 [1.0000000e+00 1.3336080e+02]
 [0.0000000e+00 1.4257200e+02]
 [2.0000000e+00 1.5015200e+02]
 [1.0000000e+00 1.5815140e+02]
 [0.0000000e+00 1.6699520e+02]
 [2.0000000e+00 1.7575440e+02]
 [1.0000000e+00 1.8472860e+02]
 [0.0000000e+00 1.9260320e+02]
 [2.0000000e+00 2.0015220e+02]
 [1.0000000e+00 2.0879960e+02]
 [0.0000000e+00 2.1692540e+02]
 [2.0000000e+00 2.2497160e+02]
 [1.0000000e+00 2.3446040e+02]
 [0.0000000e+00 2.4346140e+02]
 [2.0000000e+00 2.5113280e+02]
 [1.0000000e+00 2.5931080e+02]
 [0.0000000e+00 2.6796820e+02]
 [2.0000000e+00 2.7659040e+02]
 [1.0000000e+00 2.8558580e+02]
 [0.0000000e+00 2.9312400e+02]
 [1.0000

In [50]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Leitura dos arquivos .txt
dados1 = pd.read_csv('MEP_microvoltsvalues_robotic_1mm.txt', header=None, names=['valor'])
dados2 = pd.read_csv('MEP_microvoltsvalues_thais_1mm.txt', header=None, names=['valor'])

# Adiciona rótulos de grupo
dados1['grupo'] = 'robotic'
dados2['grupo'] = 'manual'

# Junta os dois dataframes
dados = pd.concat([dados1, dados2], ignore_index=True)

# Plot do violin plot
sns.violinplot(x='grupo', y='valor', data=dados)
plt.title("Distribuição dos MEPs por condição de posicionamento")
plt.xlabel("Condição")
plt.ylabel("Valor (µV)")
plt.show()

